In [18]:
from collections import deque
import heapq
from typing import Dict, List, Tuple, Optional, Set

Node = str
Cost = float
Adj = Dict[Node, List[Tuple[Node, Cost]]]

class GraphSearch:
    def __init__(self) -> None:
        self.g: Adj = {}

    def add_edge(self, u: Node, v: Node, w: Cost = 1.0) -> None:
        self.g.setdefault(u, []).append((v, w))
        self.g.setdefault(v, []).append((u, w))

    
    #Reconstruct the path
    @staticmethod
    def reconstruct(parent: Dict[Node, Optional[Node]], goal: Node) -> List[Node]:

        path: List[Node] = []
        cur: Optional[Node] = goal
        while cur is not None:
            path.append(cur)
            cur = parent[cur]
        path.reverse()
        return path

    def neighbors(self, u: Node) -> List[Tuple[Node, Cost]]:
        return self.g.get(u, [])

    #BFS
    def bfs(self, start: Node, goal: Node) -> Tuple[List[Node], List[Node]]:

        if start not in self.g or goal not in self.g:
            return [], []

        q: deque[Node] = deque([start])
        visited: Set[Node] = {start}
        parent: Dict[Node, Optional[Node]] = {start: None}
        expanded: List[Node] = []

        while q:
            u = q.popleft()
            expanded.append(u)
            if u == goal:
                return self.reconstruct(parent, goal), expanded
                
            for v, _ in self.neighbors(u):
                if v not in visited:
                    visited.add(v)
                    parent[v] = u
                    q.append(v)
        return [], expanded



    #Uniform cost search
    def ucs(self, start: Node, goal: Node) -> Tuple[List[Node], List[Node], Optional[Cost]]:
        
        if start not in self.g or goal not in self.g:
            return [], [], None

        heap: List[Tuple[Cost, Node]] = [(0.0, start)]
        best: Dict[Node, Cost] = {start: 0.0}
        parent: Dict[Node, Optional[Node]] = {start: None}
        expanded: List[Node] = []

        while heap:
            g_u, u = heapq.heappop(heap)
            if g_u != best.get(u, float("inf")):
                continue
            expanded.append(u)
            if u == goal:
                return self.reconstruct(parent, goal), expanded, g_u
            for v, w in self.g.get(u, []):
                new_g = g_u + w
                if new_g < best.get(v, float("inf")):
                    best[v] = new_g
                    parent[v] = u
                    heapq.heappush(heap, (new_g, v))
        return [], expanded, None



    #Search results output
    @staticmethod
    def print_result(title: str, path: List[Node], expanded: List[Node], cost: Optional[Cost] = None) -> None:
        print(f'{title}: ')
        print("Expanded:", " -> ".join(expanded) if expanded else "-")
        if path:
            print("Path:", " -> ".join(path))
            if cost is None:
                print("Path length (edges):", max(0, len(path) - 1))
            else:
                print("Total cost:", cost)
        else:
            print("No path found.")


if __name__ == "__main__":
    gs = GraphSearch()

    gs.add_edge("A", "B", 1)
    gs.add_edge("A", "C", 4)
    gs.add_edge("B", "D", 2)
    gs.add_edge("C", "D", 1)
    gs.add_edge("D", "E", 3)
    gs.add_edge("B", "E", 5)
    gs.add_edge("C", "E", 1)
    gs.add_edge("E", "F", 7)
    gs.add_edge("D", "G", 2)
    gs.add_edge("E", "H", 2)
    gs.add_edge("G", "H", 2)

    start, goal = "A", "H"

    path_bfs, expanded_bfs = gs.bfs(start, goal)
    gs.print_result("BFS", path_bfs, expanded_bfs)

    print('\n')

    path_ucs, expanded_ucs, cost_ucs = gs.ucs(start, goal)
    gs.print_result("UCS", path_ucs, expanded_ucs, cost_ucs)


BFS: 
Expanded: A -> B -> C -> D -> E -> G -> F -> H
Path: A -> B -> E -> H
Path length (edges): 3


UCS: 
Expanded: A -> B -> D -> C -> E -> G -> H
Path: A -> C -> E -> H
Total cost: 7.0
